In [12]:
import pandas as pd
import numpy as np
import os
from util import HISTORICAL_FILE_JOIN, RESOURCE_DIR 
#pip install mlxtend
from mlxtend.frequent_patterns import apriori, association_rules

Reads projects from Excel file

In [2]:
#df = pd.read_excel("historical_join.xlsx", keep_default_na=False)
df = pd.read_excel(HISTORICAL_FILE_JOIN, keep_default_na=False)

len(df)
df.head()

,PROJECTS,COMMITS,Memcached,Microsoft Azure Table Storage,FileMaker,Riak KV,Hive,Redis,Aerospike,Google Cloud Datastore,...,Teradata,PostGIS,Etcd,MySQL,Join_Ignite-Sql,Join_Ignite-NoSql,Join_MarkLogic-Sql,Join_MarkLogic-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,Activiti,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,Activiti,2,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,Activiti,3,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,Activiti,4,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,Activiti,5,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


Deleting PROJECTS and COMMITS columns to work with one hot values

In [3]:
df.drop(["PROJECTS", "COMMITS"], axis=1, inplace=True)
df

,Memcached,Microsoft Azure Table Storage,FileMaker,Riak KV,Hive,Redis,Aerospike,Google Cloud Datastore,MariaDB,Neo4j,...,Teradata,PostGIS,Etcd,MySQL,Join_Ignite-Sql,Join_Ignite-NoSql,Join_MarkLogic-Sql,Join_MarkLogic-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3265,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3266,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3267,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3268,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


Executing an a priori function to extract common itemsets for association rule acquisition
Support = 0,01 corresponds to 32.7 tuples from our dataset (3270 records).
This means that only item sets that occurred more than 32.7 times in our data will be considered. 

In [4]:
frq_items = apriori(df, min_support = 0.01, use_colnames = True)

Collecting the inferred rules in a dataframe whose lift value is above 1.

In [7]:
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)

# Sorting the rules by the lift measure 
rules = rules.sort_values('lift', ascending =False)
print(rules.head())

                                            antecedents  \
119454              (MariaDB, Oracle, SQLite, Firebird)   
146715       (SQLite, Oracle, MySQL, MariaDB, Firebird)   
76092               (MariaDB, Oracle, SQLite, Firebird)   
145695  (PostgreSQL, SQLite, Oracle, MariaDB, Firebird)   
150391              (MariaDB, Firebird, Oracle, SQLite)   

                                              consequents  antecedent support  \
119454     (IBM DB2, Informix, MS SQL Server, PostgreSQL)            0.010398   
146715             (IBM DB2, Informix, H2, MS SQL Server)            0.010398   
76092                  (IBM DB2, Informix, MS SQL Server)            0.010398   
145695             (IBM DB2, Informix, H2, MS SQL Server)            0.010398   
150391  (IBM DB2, MySQL, MS SQL Server, Informix, Post...            0.010398   

        consequent support   support  confidence       lift  leverage  \
119454             0.01315  0.010398         1.0  76.046512  0.010261   
146715      

Showing all generated association rules

In [8]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
119454,"(MariaDB, Oracle, SQLite, Firebird)","(IBM DB2, Informix, MS SQL Server, PostgreSQL)",0.010398,0.013150,0.010398,1.000000,76.046512,0.010261,inf
146715,"(SQLite, Oracle, MySQL, MariaDB, Firebird)","(IBM DB2, Informix, H2, MS SQL Server)",0.010398,0.013150,0.010398,1.000000,76.046512,0.010261,inf
76092,"(MariaDB, Oracle, SQLite, Firebird)","(IBM DB2, Informix, MS SQL Server)",0.010398,0.013150,0.010398,1.000000,76.046512,0.010261,inf
145695,"(PostgreSQL, SQLite, Oracle, MariaDB, Firebird)","(IBM DB2, Informix, H2, MS SQL Server)",0.010398,0.013150,0.010398,1.000000,76.046512,0.010261,inf
150391,"(MariaDB, Firebird, Oracle, SQLite)","(IBM DB2, MySQL, MS SQL Server, Informix, Post...",0.010398,0.013150,0.010398,1.000000,76.046512,0.010261,inf
...,...,...,...,...,...,...,...,...,...
58,(Google Cloud Datastore),(MySQL),0.040367,0.251376,0.019878,0.492424,1.958914,0.009730,1.474901
282,(Ehcache),(SQLite),0.109174,0.067890,0.013456,0.123249,1.815429,0.006044,1.063142
283,(SQLite),(Ehcache),0.067890,0.109174,0.013456,0.198198,1.815429,0.006044,1.111030
36,(Redis),(SQLite),0.096942,0.067890,0.011621,0.119874,1.765709,0.005039,1.059064


Counting the size of each rule's itemsets 

In [9]:
rules['antecedent_len'] = rules['antecedents'].apply(lambda x: len(x))
rules['consequente_len'] = rules['consequents'].apply(lambda x: len(x))

Filtering only the rules of size 2, that is, 1 antecedent and 1 consequent. 

In [10]:
rules_filter = rules[(rules['antecedent_len'] == 1) & (rules['consequente_len'] == 1)]

Showing all filtered rules.

In [11]:
rules_filter

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequente_len
167,(Firebird),(SapHana),0.016208,0.013456,0.011315,0.698113,51.882504,0.011097,3.267928,1,1
166,(SapHana),(Firebird),0.013456,0.016208,0.011315,0.840909,51.882504,0.011097,6.183836,1,1
221,(Informix),(Firebird),0.018654,0.016208,0.014067,0.754098,46.526446,0.013765,4.000754,1,1
220,(Firebird),(Informix),0.016208,0.018654,0.014067,0.867925,46.526446,0.013765,7.430188,1,1
169,(Informix),(SapHana),0.018654,0.013456,0.011009,0.590164,43.859911,0.010758,2.407168,1,1
...,...,...,...,...,...,...,...,...,...,...,...
58,(Google Cloud Datastore),(MySQL),0.040367,0.251376,0.019878,0.492424,1.958914,0.009730,1.474901,1,1
282,(Ehcache),(SQLite),0.109174,0.067890,0.013456,0.123249,1.815429,0.006044,1.063142,1,1
283,(SQLite),(Ehcache),0.067890,0.109174,0.013456,0.198198,1.815429,0.006044,1.111030,1,1
36,(Redis),(SQLite),0.096942,0.067890,0.011621,0.119874,1.765709,0.005039,1.059064,1,1


Generating the dataframe with the filtered rules.

In [14]:
HISTORICAL_RULES_FILE = RESOURCE_DIR + os.sep + 'historical_rules.xlsx'
rules_filter = rules_filter.to_excel(HISTORICAL_RULES_FILE, index=False)